<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Static-Schedule" data-toc-modified-id="Static-Schedule-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Static Schedule</a></span><ul class="toc-item"><li><span><a href="#Homogeneous-Exponential-Case" data-toc-modified-id="Homogeneous-Exponential-Case-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Homogeneous Exponential Case</a></span></li><li><span><a href="#Heterogeneous-Exponential-Case" data-toc-modified-id="Heterogeneous-Exponential-Case-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Heterogeneous Exponential Case</a></span></li><li><span><a href="#Phase-Type-Case" data-toc-modified-id="Phase-Type-Case-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Phase-Type Case</a></span><ul class="toc-item"><li><span><a href="#Phase-Type-Fit" data-toc-modified-id="Phase-Type-Fit-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Phase-Type Fit</a></span></li><li><span><a href="#Weighted-Erlang-Distribution" data-toc-modified-id="Weighted-Erlang-Distribution-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Weighted Erlang Distribution</a></span></li><li><span><a href="#Hyperexponential-Distribution" data-toc-modified-id="Hyperexponential-Distribution-1.3.3"><span class="toc-item-num">1.3.3&nbsp;&nbsp;</span>Hyperexponential Distribution</a></span></li></ul></li></ul></li></ul></div>

# Static Schedule

## Homogeneous Exponential Case

## Heterogeneous Exponential Case

## Phase-Type Case

### Phase-Type Fit

### Weighted Erlang Distribution

### Hyperexponential Distribution